In [ ]:
!pip install ipyplot
!pip install Pillow
!pip install split-folders

In [ ]:
import os
import glob
import cv2
import splitfolders
import numpy as np
import shutil
import ipyplot
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.force_b64 = True
import matplotlib.pyplot as plt
import keras
from sklearn.metrics import classification_report,confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import tensorflow_hub as hub
from tensorflow import keras
from tqdm import tqdm_notebook


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


# Optimized Mobilenet-V2

In [ ]:
train_path='/content/drive/MyDrive/0R&D/newtom/Dataset/train'
validation_data='/content/drive/MyDrive/0R&D/newtom/Dataset/val'
test_path='/content/drive/MyDrive/0R&D/newtom/Dataset/test30'

In [ ]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size=(224,224),batch_size=32)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=validation_data,target_size=(224,224),batch_size=32)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size=(224,224),batch_size=10,shuffle=True)

Found 9396 images belonging to 10 classes.
Found 1030 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


In [ ]:
mobile=keras.applications.mobilenet.MobileNet()

In [ ]:
MobileNetV2=tf.keras.applications.mobilenet_v2.MobileNetV2
model_arch=MobileNetV2()
model_arch.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
import tensorflow_hub as hub
feature_extractor = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
model2 = tf.keras.Sequential([feature_extractor])
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [ ]:
for image_batch, label_batch in train_batches:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 10)


In [ ]:
model2 = tf.keras.Sequential([feature_extractor ])
model2.add(tf.keras.layers.Dense(10))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 2270794 (8.66 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
model2.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [ ]:
type(mobile)

keras.src.engine.functional.Functional

In [ ]:
model2=keras.Sequential()
for layer in mobile.layers[:-1]:
  model2.add(layer)

In [ ]:
for layer in model2.layers:
  layer.trainable=True

In [ ]:
model2.add(keras.layers.Dense(10,activation='Softmax'))

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128       
 zation)                                                         
                                                      

In [ ]:
model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history2 = model2.fit(train_batches, epochs = 4,validation_data=valid_batches, steps_per_epoch =157,validation_steps = 2)

Epoch 1/4
157/157 [==============================] - 77s 271ms/step - loss: 1.4204 - accuracy: 0.8045 - val_loss: 31.1916 - val_accuracy: 0.2969
Epoch 2/4
157/157 [==============================] - 36s 232ms/step - loss: 0.2462 - accuracy: 0.9397 - val_loss: 1.6565 - val_accuracy: 0.8281
Epoch 3/4
157/157 [==============================] - 32s 205ms/step - loss: 0.1620 - accuracy: 0.9554 - val_loss: 0.5545 - val_accuracy: 0.8750
Epoch 4/4
157/157 [==============================] - 30s 191ms/step - loss: 0.1225 - accuracy: 0.9635 - val_loss: 0.3874 - val_accuracy: 0.8906


In [ ]:
for image_batch, label_batch in train_batches:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 10)


In [ ]:
predicted_batch = model2.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
label_id = np.argmax(label_batch, axis=-1)

1/1 [==============================] - 1s 1s/step


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_id, predicted_id, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.67      1.00      0.80         4
           2       0.00      0.00      0.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         2

    accuracy                           0.94        32
   macro avg       0.87      0.90      0.88        32
weighted avg       0.90      0.94      0.91        32




# RESNET_50

In [ ]:
IMG_HEIGHT = 128
IMG_WIDTH = 128
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/0R&D/newtom/Dataset/train",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=64
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/0R&D/newtom/Dataset/test30",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=10)
val_ds = tf.keras.preprocessing.image_dataset_from_directory("/content/drive/MyDrive/0R&D/newtom/Dataset/val",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=64)

Found 9396 files belonging to 10 classes.
Found 300 files belonging to 10 classes.
Found 1030 files belonging to 10 classes.


In [ ]:
class_names = train_ds.class_names
print(class_names)
train_ds

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(4, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(keras.applications.ResNet50(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(64, "relu"))
model.add(keras.layers.Dense(10, "softmax"))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "adam",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(train_ds,validation_data=val_ds,epochs=10, batch_size=64, verbose=1)

Epoch 1/10
157/157 [==============================] - 100s 246ms/step - loss: 0.5174 - accuracy: 0.8347 - val_loss: 20.0940 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 38s 231ms/step - loss: 0.2316 - accuracy: 0.9228 - val_loss: 16.3919 - val_accuracy: 0.1000
Epoch 3/10
157/157 [==============================] - 37s 230ms/step - loss: 0.1221 - accuracy: 0.9597 - val_loss: 5.6278 - val_accuracy: 0.1000
Epoch 4/10
157/157 [==============================] - 37s 230ms/step - loss: 0.1058 - accuracy: 0.9661 - val_loss: 11.6202 - val_accuracy: 0.0940
Epoch 5/10
157/157 [==============================] - 37s 233ms/step - loss: 0.0791 - accuracy: 0.9747 - val_loss: 3.6966 - val_accuracy: 0.1570
Epoch 6/10
157/157 [==============================] - 38s 234ms/step - loss: 0.0756 - accuracy: 0.9740 - val_loss: 2.8070 - val_accuracy: 0.4490
Epoch 7/10
157/157 [==============================] - 38s 232ms/step - loss: 0.0564 - accuracy: 0.9821 - val_loss: 2.4252 - va

In [ ]:
actual_label = []
pred_label = []
for img, label in train_ds.take(1):
    pred = model.predict(img)
    pred = np.argmax(pred, axis=1)
    print(classification_report(label,pred))

2/2 [==============================] - 0s 45ms/step
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      1.00      1.00         6
           2       1.00      0.75      0.86         4
           3       1.00      0.88      0.93         8
           4       0.88      1.00      0.93         7
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         7

    accuracy                           0.97        64
   macro avg       0.98      0.96      0.97        64
weighted avg       0.97      0.97      0.97        64



***DENSENET-169***

In [ ]:
class_names = train_ds.class_names
print(class_names)
train_ds

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
model2 = keras.models.Sequential()
model2.add(keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model2.add(keras.applications.DenseNet169(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model2.add(keras.layers.GlobalAveragePooling2D())
model2.add(keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"))
model2.add(keras.layers.Dense(64, "relu"))
model2.add(keras.layers.Dense(10, "softmax"))



In [ ]:
model2.compile(loss="sparse_categorical_crossentropy",
              optimizer = "adam",metrics=["accuracy"])

In [ ]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 densenet169 (Functional)    (None, 4, 4, 1664)        12642880  
                                                                 
 global_average_pooling2d_5  (None, 1664)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_11 (Dense)            (None, 128)               213120    
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 10)                650       
                                                      

In [ ]:
hist = model2.fit(train_ds,validation_data=val_ds,epochs=5, batch_size=64, verbose=1)

Epoch 1/5
157/157 [==============================] - 53s 330ms/step - loss: 0.0273 - accuracy: 0.9908 - val_loss: 1.6614 - val_accuracy: 0.7119
Epoch 2/5
157/157 [==============================] - 49s 309ms/step - loss: 0.0317 - accuracy: 0.9904 - val_loss: 0.2430 - val_accuracy: 0.9406
Epoch 3/5
157/157 [==============================] - 49s 310ms/step - loss: 0.0342 - accuracy: 0.9912 - val_loss: 2.2207 - val_accuracy: 0.5941
Epoch 4/5
157/157 [==============================] - 49s 309ms/step - loss: 0.0207 - accuracy: 0.9938 - val_loss: 0.3149 - val_accuracy: 0.9277
Epoch 5/5
157/157 [==============================] - 50s 311ms/step - loss: 0.0076 - accuracy: 0.9971 - val_loss: 0.0941 - val_accuracy: 0.9743


In [ ]:
for img, label in train_ds.take(1):
    print(label, len(label))
    break

tf.Tensor(
[8 4 7 4 7 5 0 0 8 6 1 3 1 7 0 8 5 4 5 3 0 1 4 0 6 3 3 3 5 8 7 6 3 9 3 0 7
 9 0 1 0 4 8 7 0 7 2 1 8 9 0 6 8 7 1 5 1 5 3 1 5 8 4 5], shape=(64,), dtype=int32) 64


In [ ]:
actual_label = []
pred_label = []
for img, label in train_ds.take(1):
    pred = model2.predict(img)
    pred = np.argmax(pred, axis=1)
    print(classification_report(label,pred))

2/2 [==============================] - 0s 65ms/step
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00         9
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         6
           9       1.00      1.00      1.00         6

    accuracy                           0.98        64
   macro avg       0.97      0.98      0.97        64
weighted avg       0.99      0.98      0.99        64

